In [1]:
#packages
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime



path = 'ImagesAttendance'  #directory of students image data with 40 student images
images = []               #creating the list of  students image path
student_Names = []           #creating the list of  students name
myList = os.listdir(path)    #listing out the files/folders in that path 
print(myList)



#Looping over the path to grab the students name
for cl in myList:
    current_Img = cv2.imread(f'{path}/{cl}')
    images.append(current_Img)
    student_Names.append(os.path.splitext(cl)[0]) #splitting the path and grabbing names, 
                                                  #eg: xyz.jpg, it splits into (xyz, .jpg)
print(student_Names)




#fuction for grabing the encodings of the images
def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList




#creates an excel file with the name of the identified student with date and timing 
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

#### FOR CAPTURING SCREEN RATHER THAN WEBCAM
# def captureScreen(bbox=(300,300,690+300,530+300)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr





encodeListKnown = findEncodings(images) #Grabbing the encodings of the faces
print('Encoding Complete')



#Below code captures and  turns the webcam on and starts to capture detect faces from the video frame
cap = cv2.VideoCapture(0)
while True:
    success, img = cap.read()  #reads the frame and gives a boolean value, TRUE = frame gets captured
    

    
    
    #Resizing the images to 1/4th of its original size
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    
    #Converting the image color from BGR to RGB as OpenCv reads image in BGR format
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    #Grabbing the co-ordinates of the the detected face
    facesCurrentFrame = face_recognition.face_locations(imgS)
    
    #Grabbing the encoding of that particular image    
    encodesCurrentFrame = face_recognition.face_encodings(imgS, facesCurrentFrame)
    
    
    #Looping over the encodings and face location for the comparison pupose
    for encodeFace, faceLoc in zip(encodesCurrentFrame, facesCurrentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        
        #Images with unique distance
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        # print(faceDis)
        
        #taking the only minimum distance 
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = student_Names[matchIndex].upper() #change the string to UpperCase
            # print(name)
            
            
            
            
            #Code for creating the bounding box with the name around the face
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    cv2.imshow('Webcam', img)
    cv2.waitKey(1)

['Aditya dhurandhar.JPG', 'Advait Shivaikar.JPG', 'Amit Kanse.JPG', 'Anjali.JPG', 'Anushka Gadgil.JPG', 'Arfat Lamboo.JPG', 'Asif.JPG', 'Chandraprakash Vishwakarma.JPG', 'Jagruti Sawalkar.JPG', 'Jasmeet Dhanota.JPG', 'Jasmit Hanspal.JPG', 'Jayesh.JPG', 'Kiran Patil.JPG', 'Kirtiraj Khape.JPG', 'Mayuresh.JPG', 'Mehtab Aalam.JPG', 'Neha Sutar.JPG', 'Onkar raut.JPG', 'Prajakta.JPG', 'Pranav Phale.JPG', 'Pranav Purushan.JPG', 'Pranita Jadhav.JPG', 'Pratima.jpg', 'Prithvi Shetty.JPG', 'Priya Jadhav.JPG', 'Reshma Sawant.JPG', 'Saarang Kulkarni.JPG', 'Sahil Kedari.JPG', 'Sangram Ghadage.JPG', 'Sheetal Patil.JPG', 'Shivam Yadav.JPG', 'Shivshankar Sahu.JPG', 'Shreyas Kulkarni.JPG', 'Shripad Joshi.JPG', 'Shubhangi B.JPG', 'Suraj Pal.JPG', 'Varad rane.JPG', 'Vinay Datir.JPG', 'Vinayak Mirjolkar.JPG', 'Vinit.JPG']
['Aditya dhurandhar', 'Advait Shivaikar', 'Amit Kanse', 'Anjali', 'Anushka Gadgil', 'Arfat Lamboo', 'Asif', 'Chandraprakash Vishwakarma', 'Jagruti Sawalkar', 'Jasmeet Dhanota', 'Jasmit Ha

PermissionError: [Errno 13] Permission denied: 'Attendance.csv'